# Find amenities around an input location(Google - Places API (New))

### Loading packages

In [17]:
import requests
import pandas as pd
import googlemaps

### Define request funtions

In [18]:
# This function returns latitude and longitude for an input address
def gecodingapi(address):
    Google_key='' # Put your google api key here
    gmaps = googlemaps.Client(key=Google_key)
    # Geocoding an address
    geocode_result = gmaps.geocode(address)
    latitude = geocode_result[0]['geometry']['location']['lat']
    longitude = geocode_result[0]['geometry']['location']['lng']
    return latitude,longitude

This function calls the Google API and returns dataframe based on your input parameters:

url: 'https://places.googleapis.com/v1/places:searchNearby'

search_type: You can find the type of search from the Google document.

search_type_name: Self-Name your search type.

maxcount: Number of reslts return(max: 20)

exclude_type:  List of types that exluded in a search(Google document)

lat: Latitude,

lon: Longitude,

radius: Radius of circle in metre,

field: Set up returning information such as Name, Address, Photo, primary type, url and so on(Google document)

*Document: https://developers.google.com/maps/documentation/places/web-service/nearby-search

*Field: https://developers.google.com/maps/documentation/places/web-service/place-types

In [19]:
# This function calls the Google API and returns dataframe based on your input fields
def google_api(url,search_type,search_type_name,maxcount,exclude_type,lat,lon,radius,field):
    # Define the API endpoint
    url = url
    # Define the payload
    Google_key='' # Put your google api key here
    if not exclude_type:
        payload = {
            "includedTypes": search_type, # List of types can be found at Table A, B
            "maxResultCount": maxcount, # Number of results returns(max: 20)
            "locationRestriction": {
                "circle": {
                    "center": {
                        "latitude": lat, # Latitude
                        "longitude": lon # Longitude
                    },
                    "radius": radius # Radius of circle in metre
                }
            }
        }
    else:
        payload = {
            "includedTypes": search_type, # List of types can be found at Table A, B
            "maxResultCount": maxcount, # Number of results returns(max: 20)
            "excludedTypes": exclude_type, # List of types that exluded in the result
            "locationRestriction": {
                "circle": {
                    "center": {
                        "latitude": lat, # Latitude
                        "longitude": lon # Longitude
                    },
                    "radius": radius # Radius of circle in metre
                }
            }
        }

    # Define the headers
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': Google_key,
        'X-Goog-FieldMask': field # Address informations
    }

    # Make the POST request
    response = requests.post(url, json=payload, headers=headers)
    places = response.json()['places']
    # Create a DataFrame
    df_search= pd.DataFrame([{
        'Searched': search_type_name,
        'Name': place.get('displayName', {}).get('text', 'N/A'),
        'Address': place['formattedAddress'],
        'Website': place.get('websiteUri', 'N/A'),
        'Google Maps': place.get('googleMapsUri', 'N/A'),
        'Types': ', '.join(place['types']),
        'Latitude': place.get('location', {}).get('latitude', 'N/A'),
        'Longitude': place.get('location', {}).get('longitude', 'N/A'),
        'Primary Type': place.get('primaryTypeDisplayName', {}).get('text', 'N/A'),
        'Photos': 'https://places.googleapis.com/v1/'+place.get('photos')[0].get('name', 'N/A')+'/media?key='+Google_key+'&maxHeightPx=600'
    } if place.get('photos') is not None else {'Searched': search_type_name,
        'Name': place.get('displayName', {}).get('text', 'N/A'),
        'Address': place['formattedAddress'],
        'Website': place.get('websiteUri', 'N/A'),
        'Google Maps': place.get('googleMapsUri', 'N/A'),
        'Types': ', '.join(place['types']),
        'Latitude': place.get('location', {}).get('latitude', 'N/A'),
        'Longitude': place.get('location', {}).get('longitude', 'N/A'),
        'Primary Type': place.get('primaryTypeDisplayName', {}).get('text', 'N/A'),
        'Photos': None}
    for place in places])
    return df_search

In [20]:
def google_api_primary(url,search_type,search_type_name,maxcount,exclude_type,lat,lon,radius,field):
    # Define the API endpoint
    url = url
    # Define the payload
    Google_key='' # Put your google api key here
    if not exclude_type:
        payload = {
            "includedPrimaryTypes": search_type,
            "maxResultCount": maxcount,
            "locationRestriction": {
                "circle": {
                    "center": {
                        "latitude": lat,
                        "longitude": lon
                    },
                    "radius": radius
                }
            }
        }
    else:
        payload = {
            "includedPrimaryTypes": search_type,
            "maxResultCount": maxcount,
            "excludedTypes": exclude_type,
            "locationRestriction": {
                "circle": {
                    "center": {
                        "latitude": lat,
                        "longitude": lon
                    },
                    "radius": radius
                }
            }
        }

    # Define the headers
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': Google_key, 
        'X-Goog-FieldMask': field
    }

    # Make the POST request
    response = requests.post(url, json=payload, headers=headers)
    # print(response.json())
    places = response.json()['places']
    
    # Create a DataFrame
    df_search= pd.DataFrame([{
        'Searched': search_type_name,
        'Name': place.get('displayName', {}).get('text', 'N/A'),
        'Address': place['formattedAddress'],
        'Website': place.get('websiteUri', 'N/A'),
        'Google Maps': place.get('googleMapsUri', 'N/A'),
        'Types': ', '.join(place['types']),
        'Latitude': place.get('location', {}).get('latitude', 'N/A'),
        'Longitude': place.get('location', {}).get('longitude', 'N/A'),
        'Primary Type': place.get('primaryTypeDisplayName', {}).get('text', 'N/A'),
        'Photos': 'https://places.googleapis.com/v1/'+place.get('photos')[0].get('name', 'N/A')+'/media?key='+Google_key+'&maxHeightPx=600'
    } if place.get('photos') is not None else {'Searched': search_type_name,
        'Name': place.get('displayName', {}).get('text', 'N/A'),
        'Address': place['formattedAddress'],
        'Website': place.get('websiteUri', 'N/A'),
        'Google Maps': place.get('googleMapsUri', 'N/A'),
        'Types': ', '.join(place['types']),
        'Latitude': place.get('location', {}).get('latitude', 'N/A'),
        'Longitude': place.get('location', {}).get('longitude', 'N/A'),
        'Primary Type': place.get('primaryTypeDisplayName', {}).get('text', 'N/A'),
        'Photos': None}
    for place in places])
    return df_search

### Example

In [25]:
url='https://places.googleapis.com/v1/places:searchNearby' # This url is for all searching nearby places
location="Ashfield, Sydney, Australia"
lat= gecodingapi(location)[0]
lon= gecodingapi(location)[1]
radius= 500.0
field= 'places.displayName,places.formattedAddress,places.types,places.primaryTypeDisplayName,places.websiteUri,places.location,places.googleMapsUri,places.primaryType,places.photos'
# Search for shopping stores around
df_shop=google_api_primary(url=url,maxcount=5,search_type=["supermarket","shopping_mall","market","liquor_store","jewelry_store","home_improvement_store","home_goods_store","grocery_store","furniture_store","gift_shop","hardware_store","clothing_store","convenience_store"],search_type_name="SHOPPING & RETAIL",exclude_type=["child_care_agency"],lat=lat,lon=lon,radius=radius,field=field)
# Search for restaurances around
df_res=google_api(url=url,maxcount=5,search_type="restaurant",search_type_name="CAFES & RESTAURANTS",exclude_type=[],lat=lat,lon=lon,radius=radius,field=field)
# Search for schools around
df_edu=google_api(url=url,maxcount=5,search_type="school",search_type_name="EDUCATION",exclude_type=["fitness_center"],lat=lat,lon=lon,radius=radius,field=field)
# Search for entertainment places around
df_lr=google_api(url=url,maxcount=5,search_type=["amusement_center", "amusement_park", "aquarium", "banquet_hall", "bowling_alley", "casino", "community_center", "convention_center", "cultural_center", "dog_park", "hiking_area", "historical_landmark", "marina", "movie_rental", "movie_theater", "national_park", "park", "tourist_attraction", "visitor_center", "zoo"],search_type_name="LEISURE & RECREATION",exclude_type="",lat=lat,lon=lon,radius=radius,field=field)
# Combine all df above
combined_df = pd.concat([df_shop, df_res,df_edu,df_lr], axis=0).reset_index().drop(["index"], axis=1)
combined_df